In [ ]:
import numpy as np

In [ ]:
with open("input.txt") as f:
    dataraw = f.read()
dataraw[-5:]

In [ ]:
dataraw = """\
1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9"""

In [ ]:
data = np.array([ list(map(lambda x:list(map(int, x.split(','))), dat.split('~'))) for dat in dataraw[:-1].split('\n') ])
nblocks = len(data)
data, nblocks

In [ ]:
def get_world(data):
    world = np.zeros(np.max(data, axis=(0,1))+1)
    print(world.shape)
    for i in range(len(data)):
        block = data[i]
        x1, y1, z1 = block[0]
        x2, y2, z2 = block[1]
        world[x1:x2+1, y1:y2+1, z1:z2+1] = i+1
    return world

In [ ]:
def fall_down(world, nblocks):
    new_world = world.copy()
    fell = 0
    for i in np.arange(1, nblocks+1):
        Bx, By, Bz = np.where(world==i)
        #print(i, Bx, By, Bz)
        if 0 in Bz:
            continue
        if np.all(new_world[(Bx, By, Bz-1)]==0) or np.all(np.isin(new_world[(Bx, By, Bz-1)], [0,i])): # block can fall down or is its own.
            #print(f"block {i} can fall down")
            fell += 1
            new_world[(Bx, By, Bz)] = 0
            new_world[(Bx, By, Bz-1)] = i
    return new_world, fell

In [ ]:
def fall_all_down(world, nblocks):
    maxit = 1000
    it = 0
    while True:
        new_world, fell = fall_down(world, nblocks)
        if not fell:
            print("no block fell down anymore")
            break
        else:
            #print("new:", new_world)
            world = new_world.copy()
        it += 1
        if it >= maxit:
            print("reached max it")
            break
    return new_world

In [ ]:
def is_disintegrateble(fallen_world, i):
    Bx, By, Bz = np.where(fallen_world==i)
    above = fallen_world[(Bx, By, np.max(Bz)+1)]
    above = above[above!=0]
    above = np.unique(above)
    #print(above)
    if len(above)==0:
        return True
    for iblock in above:
        Cx, Cy, Cz = np.where(fallen_world==iblock)
        below = fallen_world[(Cx, Cy, np.min(Cz)-1)]
        below = below[below != 0]
        below = below[below != i]
        #print(iblock, below)
        if len(below): # if there is an index below that is not the current block index, it is still supported
            return True
    else:
        return False

In [ ]:
world = get_world(data)
world

In [ ]:
new_world, fell = fall_down(world, nblocks)
new_world

In [ ]:
fallen_world = fall_all_down(world, nblocks)

In [ ]:
nD = 0
for i in np.arange(1, nblocks+1):
    isD = is_disintegrateble(fallen_world, i)
    if isD:
        nD += 1
nD

In [ ]:
# 568 is too high.